Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Using the AWS Batch Architecture for AlphaFold

This notebook allows you to predict protein structures using AlphaFold on AWS Batch. 

**Differences to AlphaFold Notebooks**

In comparison to AlphaFold v2.1.0, this notebook uses AWS Batch to submit protein analysis jobs to a scalable compute cluster. The accuracy should be the same as if you ran it locally. However, by using HPC services like AWS Batch and Amazon FSx for Lustre, we can support parallel job execution and optimize the resources for each run.

**Citing this work**

Any publication that discloses findings arising from using this notebook should [cite](https://github.com/deepmind/alphafold/#citing-this-work) the [AlphaFold paper](https://doi.org/10.1038/s41586-021-03819-2).

**Licenses**

Please refer to the `LICENSE` and `THIRD-PARTY-NOTICES` file for more information about third-party software/licensing.

## Table of Contents
0. [Install Dependencies](#0.-Install-Dependencies)
1. [Run a monomer analysis job](#1.-Run-a-monomer-analysis-job)
2. [Run a multimer analysis job](#2.-Run-a-multimer-analysis-job) 
3. [Analyze multiple proteins](#3.-Analyze-multiple-proteins)

## 0. Install Dependencies

In [ ]:
# %pip install -U biopython

In [1]:
# Import required Python packages
# from Bio import SeqIO
# from Bio.SeqRecord import SeqRecord
import boto3
from datetime import datetime
# from IPython import display
# from nbhelpers import nbhelpers
import os
import pandas as pd
import sagemaker
from time import sleep

pd.set_option("max_colwidth", None)

In [2]:
# Get client informatiion
boto_session = boto3.session.Session(profile_name="bloyal+proteinfolding-Admin")
sm_session = sagemaker.session.Session(boto_session)
region = boto_session.region_name
s3_client = boto_session.client("s3", region_name=region)
batch_client = boto_session.client("batch")

S3_BUCKET = sm_session.default_bucket()
print(f" S3 bucket name is {S3_BUCKET}")

 S3 bucket name is sagemaker-us-east-2-032243382548


## 1. Run a monomer analysis job

Provide sequences for analysis

In [3]:
from batchfold.batchfold_target import BatchFoldTarget

target = BatchFoldTarget(
    target_id="T1084", 
    s3_bucket=S3_BUCKET,
    s3_base_prefix = "T1084",
    boto_session=boto_session
    ).add_sequence(
        seq="MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH",
        description="Meio, Meiothermus silvanus, 73 residues|",
    ).upload_fasta()

In [10]:
from batchfold.batchfold_environment import BatchFoldEnvironment
from batchfold.jackhmmer_job import JackhmmerJob
from batchfold.openfold_job import OpenFoldJob

batch_environment = BatchFoldEnvironment(boto_session = boto_session)

jackhmmer_job = JackhmmerJob(
    job_name = target.target_id + "_JackhmmerJob_" + datetime.now().strftime("%Y%m%d%s"),
    target_id = target.target_id,
    fasta_s3_uri = target.get_fasta_s3_uri(),
    output_s3_uri = target.get_msas_s3_uri(),
    use_small_bfd = True,
    boto_session = boto_session,
    cpu = 4,
    memory = 24
)

openfold_job = OpenFoldJob(
    job_name = target.target_id + "_OpenFoldJob_" + datetime.now().strftime("%Y%m%d%s"),
    target_id = target.target_id,
    fasta_s3_uri = target.get_fasta_s3_uri(),
    msa_s3_uri = target.get_msas_s3_uri(),
    output_s3_uri = target.get_predictions_s3_uri(),
    use_precomputed_msas = True,
    config_preset = "finetuning_ptm",
    openfold_checkpoint_path = "openfold_params/finetuning_ptm_1.pt",
    save_outputs = True,
    boto_session = boto_session,
    cpu = 4,
    memory = 8,
    gpu = 1
)

output = batch_environment \
    .submit_job(jackhmmer_job, job_queue_name="GravitonSpotJobQueue") \
    .submit_job(openfold_job, job_queue_name="G4dnJobQueue", dependent=True)

In [11]:
jackhmmer_job.describe_job()

[{'jobArn': 'arn:aws:batch:us-east-2:032243382548:job/2393fc52-a28d-4691-8334-5d627cd151a0',
  'jobName': 'T1084_JackhmmerJob_202207141657836781',
  'jobId': '2393fc52-a28d-4691-8334-5d627cd151a0',
  'jobQueue': 'arn:aws:batch:us-east-2:032243382548:job-queue/GravitonSpotJobQueue-Ek1mq0xWLn72wHtg',
  'status': 'SUBMITTED',
  'attempts': [],
  'createdAt': 1657836781378,
  'retryStrategy': {'attempts': 3, 'evaluateOnExit': []},
  'dependsOn': [],
  'jobDefinition': 'arn:aws:batch:us-east-2:032243382548:job-definition/MSAJobDefinition-a3676b5d16ced49:1',
  'parameters': {},
  'container': {'image': '032243382548.dkr.ecr.us-east-2.amazonaws.com/batch-protein-folding-2206024-container-1vgix207fj09z-msacontainerregistry-ijkx3xxwmgxd:latest',
   'command': ['aws s3 cp s3://sagemaker-us-east-2-032243382548/T1084/fastas/T1084.fasta /tmp/msa/fasta/ && python3 /opt/msa/create_alignments.py /tmp/msa/fasta --output_dir /tmp/msa/output --cpu 4 --mgnify_database_path /database/mgnify/mgy_clusters_20

Submit two Batch jobs, the first one for the data prep and the second one (dependent on the first) for the structure prediction.

In [ ]:
job_name = status_1["jobName"]
# job_name = "1234567890"  # You can also provide the name of a previous job here
nbhelpers.download_results(bucket=S3_BUCKET, job_name=job_name, local="data")

View MSA information

In [ ]:
nbhelpers.plot_msa_output_folder(
    path=f"data/{job_name}/{job_name}/msas", id=input_ids[0]
)

View predicted structure

In [ ]:
pdb_path = os.path.join(f"data/{job_name}/{job_name}/ranked_0.pdb")
print("Default coloring is by plDDT")
nbhelpers.display_structure(pdb_path)

print("Can also use rainbow coloring")
nbhelpers.display_structure(pdb_path, color="rainbow")

## 2. Run a multimer analysis job

Provide sequences for analysis

In [ ]:
## Enter the amino acid sequence to fold

id_1 = "5ZNG_1"
sequence_1 = "MDLSNMESVVESALTGQRTKIVVKVHMPCGKSRAKAMALAASVNGVDSVEITGEDKDRLVVVGRGIDPVRLVALLREKCGLAELLMVELVEKEKTQLAGGKKGAYKKHPTYNLSPFDYVEYPPSAPIMQDINPCSTM"

id_2 = "5ZNG_2"
sequence_2 = (
    "MAWKDCIIQRYKDGDVNNIYTANRNEEITIEEYKVFVNEACHPYPVILPDRSVLSGDFTSAYADDDESCYRHHHHHH"
)

# Add additional sequences, if necessary

input_ids = (
    id_1,
    id_2,
)
input_sequences = (
    sequence_1,
    sequence_2,
)

DB_PRESET = "reduced_dbs"

input_sequences, model_preset = nbhelpers.validate_input(input_sequences)
sequence_length = len(max(input_sequences))

# Upload input file to S3
job_name = nbhelpers.create_job_name()
object_key = nbhelpers.upload_fasta_to_s3(
    input_sequences, input_ids, S3_BUCKET, job_name, region=region
)

Submit batch jobs

In [ ]:
# Define resources for data prep and prediction steps
if DB_PRESET == "reduced_dbs":
    prep_cpu = 4
    prep_mem = 16
    prep_gpu = 0

else:
    prep_cpu = 16
    prep_mem = 32
    prep_gpu = 0

if sequence_length < 700:
    predict_cpu = 4
    predict_mem = 16
    predict_gpu = 1
else:
    predict_cpu = 16
    predict_mem = 64
    predict_gpu = 1

step_1_response = nbhelpers.submit_batch_alphafold_job(
    job_name=str(job_name),
    fasta_paths=object_key,
    output_dir=job_name,
    db_preset=DB_PRESET,
    model_preset=model_preset,
    s3_bucket=S3_BUCKET,
    cpu=prep_cpu,
    memory=prep_mem,
    gpu=prep_gpu,
    use_spot_instances=True,
    run_features_only=True,
)

print(f"Job ID {step_1_response['jobId']} submitted")

step_2_response = nbhelpers.submit_batch_alphafold_job(
    job_name=str(job_name),
    fasta_paths=object_key,
    output_dir=job_name,
    db_preset=DB_PRESET,
    model_preset=model_preset,
    s3_bucket=S3_BUCKET,
    cpu=predict_cpu,
    memory=predict_mem,
    gpu=predict_gpu,
    features_paths=os.path.join(job_name, job_name, "features.pkl"),
    depends_on=step_1_response["jobId"],
)

print(f"Job ID {step_2_response['jobId']} submitted")

Check status of jobs

In [ ]:
status_1 = nbhelpers.get_batch_job_info(step_1_response["jobId"])
status_2 = nbhelpers.get_batch_job_info(step_2_response["jobId"])

print(f"Data prep job {status_1['jobName']} is in {status_1['status']} status")
print(f"Predict job {status_2['jobName']} is in {status_2['status']} status")

Download results from S3

In [ ]:
job_name = status_1["jobName"]
# job_name = "1234567890"  # You can also provide the name of a previous job here
nbhelpers.download_results(bucket=S3_BUCKET, job_name=job_name, local="data")

View MSA information

In [ ]:
nbhelpers.plot_msa_output_folder(
    path=f"data/{job_name}/{job_name}/msas", id=input_ids[0]
)

View predicted structure

In [ ]:
pdb_path = os.path.join(f"data/{job_name}/{job_name}/ranked_0.pdb")
print("Can also color by chain")
nbhelpers.display_structure(pdb_path, chains=2, color="chain")

## 3. Analyze multiple proteins

Download and process CASP14 sequences

In [ ]:
!wget "https://predictioncenter.org/download_area/CASP14/sequences/casp14.seq.txt" 

In [ ]:
!sed '137,138d' "casp14.seq.txt" > "casp14_dedup.fa" # Remove duplicate entry for T1085

In [ ]:
casp14_iterator = SeqIO.parse("casp14_dedup.fa", "fasta")
casp14_df = pd.DataFrame(
    (
        (record.id, record.description, len(record), record.seq)
        for record in casp14_iterator
    ),
    columns=["id", "description", "length", "seq"],
).sort_values(by="length")

Display information about CASP14 proteins

In [ ]:
with pd.option_context("display.max_rows", None):
    display.display(casp14_df.loc[:, ("id", "description")])

Plot distribution of the protein lengths

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
plt.hist(casp14_df.length, bins=50)
plt.ylabel("Sample count")
plt.xlabel("Residue count")
plt.title("CASP-14 Protein Length Distribution")
plt.show()

Submit analysis jobs for a subset of CASP14 proteins

In [ ]:
protein_count = (
    5  # Change this to analyze a larger number of CASP14 targets, smallest to largest
)
job_name_list = []

for row in casp14_df[:protein_count].itertuples(index=False):
    record = SeqRecord(row.seq, id=row.id, description=row.description)
    print(f"Protein sequence for analysis is \n{record.description}")
    sequence_length = len(record.seq)
    print(f"Sequence length is {sequence_length}")
    print(record.seq)

    input_ids = (record.id,)
    input_sequences = (str(record.seq),)
    DB_PRESET = "reduced_dbs"

    input_sequences, model_preset = nbhelpers.validate_input(input_sequences)

    # Upload input file to S3
    job_name = nbhelpers.create_job_name()
    object_key = nbhelpers.upload_fasta_to_s3(
        input_sequences, input_ids, S3_BUCKET, job_name, region=region
    )

    # Define resources for data prep and prediction steps
    if DB_PRESET == "reduced_dbs":
        prep_cpu = 4
        prep_mem = 16
        prep_gpu = 0

    else:
        prep_cpu = 16
        prep_mem = 32
        prep_gpu = 0

    if sequence_length < 700:
        predict_cpu = 4
        predict_mem = 16
        predict_gpu = 1
    else:
        predict_cpu = 16
        predict_mem = 64
        predict_gpu = 1

    step_1_response = nbhelpers.submit_batch_alphafold_job(
        job_name=str(job_name),
        fasta_paths=object_key,
        output_dir=job_name,
        db_preset=DB_PRESET,
        model_preset=model_preset,
        s3_bucket=S3_BUCKET,
        cpu=prep_cpu,
        memory=prep_mem,
        gpu=prep_gpu,
        run_features_only=True,
    )

    print(f"Job ID {step_1_response['jobId']} submitted")

    step_2_response = nbhelpers.submit_batch_alphafold_job(
        job_name=str(job_name),
        fasta_paths=object_key,
        output_dir=job_name,
        db_preset=DB_PRESET,
        model_preset=model_preset,
        s3_bucket=S3_BUCKET,
        cpu=predict_cpu,
        memory=predict_mem,
        gpu=predict_gpu,
        features_paths=os.path.join(job_name, job_name, "features.pkl"),
        depends_on=step_1_response["jobId"],
    )

    print(f"Job ID {step_2_response['jobId']} submitted")
    sleep(1)